In [1]:
%load_ext autoreload
%autoreload 2

import logging
from functools import partial

import pandas as pd

from etf_scraper import ETFScraper, load_listings
from etf_scraper.utils import get_interval_query_dates
from etf_scraper.storage import query_range, save_func

logger = logging.getLogger()
logger.setLevel(logging.INFO)

query_dates = get_interval_query_dates('2020-01-01', '2023-01-20', False, False)
save_func_ = partial(save_func, out_dir="/tmp/out/")

listings = load_listings()

!mkdir /tmp/out/

mkdir: cannot create directory ‘/tmp/out/’: File exists


In [2]:
query_tickers = listings.ticker.unique()
query_dates = [None]

In [3]:
# query_tickers = ["IVV"]
a = query_range(query_dates, query_tickers, ETFScraper(), save_func_)

INFO:etf_scraper.api:Querying for EIDO holdings as of latest from IShares
INFO:etf_scraper.api:Querying for MCHI holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BKTSX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for IWV holdings as of latest from IShares
INFO:etf_scraper.api:Querying for MUB holdings as of latest from IShares
INFO:etf_scraper.api:Querying for IXC holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BSPIX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BSPPX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for ISTB holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BMED holdings as of latest from IShares
INFO:etf_scraper.scrapers:Querying IShares for EIDO holdings as of None
INFO:etf_scraper.scrapers:Querying IShares for MCHI holdings as of None
INFO:etf_scraper.scrapers:Querying IShares for BKTSX holdings as of None
INFO:etf_scraper.scrapers:Querying IShares 

In [35]:
df = pd.DataFrame(a).T.rename_axis(index=['ticker', 'date']).reset_index()
df_ = df.merge(listings, on='ticker')
df['error_class'].value_counts(dropna=False)

NaN                      713
ValueError               287
InvalidParameterError    191
KeyError                  38
EmptyDataError             6
HTTPError                  5
Name: error_class, dtype: int64

In [33]:
df_[~df_['error'].isna()][['provider', "asset_class"]].value_counts(dropna=False).sort_index()

provider  asset_class   
IShares   Commodity           3
          Equity             39
          Fixed Income      126
          Multi Asset         9
          Real Estate         3
Invesco   NaN                86
SSGA      Alternative         4
          Equity             28
          Fixed Income       38
          NaN                69
Vanguard  Balanced Funds     10
          Bond Funds         55
          Stock Funds        57
dtype: int64

In [5]:
df_[
    (~df_['error'].isna())&(df_['provider']=='SSGA')&(df_['asset_class']=='Equity')
    # &(df_['ticker']=='SPTM')
    ]#['error'].iloc[0]

NameError: name 'df_' is not defined